In [63]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [64]:
import random

random.seed(0)

Sobre la data:

Desde VSE++ (https://github.com/fartashf/vsepp)

Particiones F30K: https://www.kaggle.com/datasets/shtvkumar/karpathy-splits

Data: https://www.kaggle.com/datasets/hsankesara/flickr-image-dataset

Extraccion con EfNetV2 L 30K (13 min) 
https://www.kaggle.com/code/mariocarlosmallea/extrac-feautures-f30k/edit/run/108300793

Extraccion MPNET 3 (min): 

Matriz en orden de aparición

## Data

In [65]:
import json as jsonmod
import numpy as np
import random
import time
import pickle
from tqdm import tqdm



In [66]:
name_data= "coco"#"coco" #"F30K"#"coco" #"F30K" #"coco"

In [67]:

if name_data=="F30K":
    path= 'F30K' 
    dataset = jsonmod.load(open(path+"/dataset_flickr30k.json", 'r'))['images']
    
else: #coco structure to (like) F30K

    path= 'COCO' 
    map_img=np.load(path+"/map_img_coco_restval.npy",allow_pickle=True)
    map_text=np.load(path+"/map_text_coco_restval.npy",allow_pickle=True)
    dataset= np.load(path+"/dataset_coco_restval.npy",allow_pickle=True)
                    
                    
all_val_indices_img,all_test_indices_img, all_val_indices_text,all_test_indices_text=[],[],[],[]

for i in range(len(dataset)):
  if dataset[i]["split"]=="val":
    all_val_indices_img.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      all_val_indices_text.append(t)


  if dataset[i]["split"]=="test":
    all_test_indices_img.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      all_test_indices_text.append(t)

print("Train-Val-Test")  
print(len(dataset), len(all_val_indices_img), len(all_test_indices_img))

                

Train-Val-Test
123287 5000 5000


dataset -> [ ... (imgid, [5 textos] )...]

In [68]:
#solo test
dataset_= []
random_train=[]
for i in range(len(dataset)):
  if dataset[i]["split"]=="test":
    dataset_.append(dataset[i])
    
  #if dataset[i]["split"]=="train" :
   # dataset_.append(dataset[i])
    
dataset= list(dataset_)


In [69]:
train_indices=[]
data_text=[] #querys train

 #dictionary ground truth retrieval img o text
all_r_img, all_r_text= {},{} #train+val+test
val_r_img, val_r_text, test_r_img, test_r_text= {},{},{},{} 

for i in range(len(dataset)):
  if dataset[i]["split"]=="train":
    train_indices.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      data_text.append(t)
      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]


  if dataset[i]["split"]=="val":
    
    for t in dataset[i]["sentids"]:
      val_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      val_r_img[t]= dataset[i]["imgid"]
        
      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]

  if dataset[i]["split"]=="test":
    
    for t in dataset[i]["sentids"]:
      test_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      test_r_img[t]= dataset[i]["imgid"]
        
      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]


print("Train-Val-Test")
print("img: ", len(train_indices),len(val_r_text.keys()), len(test_r_text.keys()))
print("text: ",len(data_text), len(val_r_img.keys()), len(test_r_img.keys()))
print("-"*20)
# captions order for modeling e.g: [id 999] ->[position 0] with .index
universe_text= data_text+ list(val_r_img.keys())+list(test_r_img.keys()) 
#img order for modeling
universe_img= train_indices+list(val_r_text.keys())+ list(test_r_text.keys())
print("All Img-Text")
print(len(universe_img),len(universe_text))


Train-Val-Test
img:  0 0 5000
text:  0 0 25000
--------------------
All Img-Text
5000 25000


In [62]:
np.save("coco_test_universe_text.npy",universe_text)
np.save("coco_test_universe_img.npy",universe_img)

In [73]:

import sklearn.preprocessing
if name_data=="F30K":
    
    textual_matrix= np.load("ALIGN/AALIGN_text_F30K_.npy")
    visual_matrix= np.load("ALIGN/AALIGN_imgs_F30K.npy")
    
else: #coco    
    textual_matrix= np.load("ALIGN/AALIGN_text_coco_.npy")
    visual_matrix= np.load("ALIGN/AALIGN_imgs_coco.npy")
    
print(textual_matrix.shape, visual_matrix.shape)


#visual_matrix=sklearn.preprocessing.normalize(visual_matrix[universe_img,:], norm='l2', axis=0) #visual_matrix[universe_img,:]/np.max(visual_matrix)
#textual_matrix=textual_matrix[universe_text,:]#/np.max(textual_matrix)
print(textual_matrix.shape, visual_matrix.shape)



(25000, 640) (5000, 640)
(25000, 640) (5000, 640)


## Evaluation

In [86]:

import concurrent.futures
from collections import defaultdict
from scipy.sparse import csr_matrix
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing
import gc


_model = None
_universe_query= None
_retrieval= None
evaluation_Q_= None
_universe_value= None
_list_rank= None
_at_k= None
evaluation_V_=None
def evaluate_model(model, nDCG, split, retrieval): #i2i
    """
    evaluate model
    """
    global _model
    global _universe_query
    global _universe_value
    global _retrieval
    global evaluation_Q_
    global _list_rank
    global _at_k
    global evaluation_V_
    
    _list_rank= []
    _retrieval=retrieval
    _model = model
    test_users = []
    results=[]
    output= []
    _at_k=10
    
    #-----------------------------------------------------------
    if _retrieval=="t2i":
      _universe_query= universe_text
      _universe_value= universe_img
      if split=="val":
        evaluation_Q_= val_r_img
        evaluation_V_= list(val_r_text.keys())
      if split=="test": #test
        evaluation_Q_= test_r_img
        evaluation_V_= list(test_r_text.keys())
      #output.append(t2i( _model,split))
    
    else:  #i2t
      _universe_query= universe_img
      _universe_value= universe_text
      if split=="val":
        evaluation_Q_= val_r_text
        evaluation_V_= list(val_r_img.keys())
      else: #test
        evaluation_Q_= test_r_text
        evaluation_V_= list(test_r_img.keys())
      #output.append(i2t( _model,split))
    """
    evaluation_Q_num = len(evaluation_Q_.keys())
    
    
    result_acumulate= np.array([0.] * 3) # 3 metrics
    for q in evaluation_Q_.keys():
        result_acumulate+=eval_one_user(q,split) 

    ret = result_acumulate / evaluation_Q_num #avg 
    ret = list(ret)

    results.append(ret)
    output.append(results)
    """

    if nDCG: #cambiar, lo dejare solo novedad
        #ndcg_i2t,ndcg_t2i= 0,0 #i2t( _model,split),0 #,t2i( _model, split)
        ndcg_i2i, ndcg_t2t= t2i( _model, split ,aux_mod=True), t2t( _model,split)
        output.append((ndcg_i2i,ndcg_t2t))
      
    
    return output



def eval_one_user(query,split):
    """
    each evaluation
    """
    
    rating = _model.predict(_universe_query.index(query), _retrieval,split)
    #gc.collect()
    if _retrieval=="t2i":
      ground_truth = [ evaluation_Q_[query] ] #one img
    else: #i2t
      ground_truth =  evaluation_Q_[query] #list
      #print(query)
    
    rank_=np.argsort(-rating)
    rank= [evaluation_V_[i]  for i in rank_[:_at_k]]


    result = []
    result.append(recall_at_k(rank, ground_truth, 1))
    result.append(recall_at_k(rank, ground_truth, 5))
    result.append(recall_at_k(rank, ground_truth, 10))

    
    return np.array(result)



def recall_at_k(rank, ground_truth, k):
    """
    Recall
    """
    #return 100 * len(set(rank[:k]) & set(ground_truth)) / len(ground_truth)
    if len(set(rank[:k]) & set(ground_truth))>=1:
        return 100
    else:
        return 0


def calculate_diversity(_list_rank, len_q):
    """ 
    max_= len(_universe_value)    
    coverage= set()
    label_binarizer = sklearn.preprocessing.LabelBinarizer()
    label_binarizer.fit(range(max_))
    list_=[]
    dict_count={}
    for rank in _list_rank: # each RANKING
        coverage= coverage | set(rank)
        list_.append( np.sum(label_binarizer.transform(rank),axis=0) )
        for i in rank:
            if i not in dict_count:
                dict_count[i]=1
            else:
                dict_count[i]+=1
    recs_content=csr_matrix(np.array(list_) )
    #calculate similarity scores for all items in list
    similarity = cosine_similarity(X=recs_content, dense_output=False)
    #get indicies for upper right triangle w/o diagonal
    upper_right = np.triu_indices(similarity.shape[0], k=1)
    diversity_cos = np.mean(similarity[upper_right])
    
    """
    _at_k=25
    dict_count={}
    print("novedad: ", _at_k, len_q )
    for rank in _list_rank: # each RANKING
        #coverage= coverage | set(rank)
        #list_.append( np.sum(label_binarizer.transform(rank),axis=0) )
        for i in rank:
            if i not in dict_count:
                dict_count[i]=1
            else:
                dict_count[i]+=1
    #novelty-diver
    nov=0
    for rank in _list_rank: # each RANKING
        self_information=0
        for i in rank:
            self_information += np.sum( -np.log2(dict_count[i]/len_q) )
        nov+=self_information/_at_k #top
        #
    
    return (0., 0., nov/len_q) #(diversity_cos, len(coverage)/max_, nov/len(evaluation_Q_))
       


In [87]:
# (C) Mathieu Blondel, November 2013
# License: BSD 3 clause
#Original https://github.com/mesnico/TERAN/blob/master/evaluate_utils/dcg.py

import numpy as np
import os

class DCG:
    def __init__(self, rank=25, relevance_methods=['rougeL'], split="val", data=name_data): # config, n_queries, split
        self.rank = rank
        self.relevance_methods = relevance_methods
        #n_queries=npts
        if data=="F30K":
            if split=="val":
                relevance_filenames= [path+"/f30k-val-rougeL.npy", path+"/f30k-val-spice.npy"] #5.140.980= 5070x1014
                self.npts= 1014 #images
            else:#test
                relevance_filenames= [path+"/f30k-test-rougeL.npy", path+"/f30k-test-spice.npy"] #500.000=5000x1000
                self.npts= 1000 #images
        else: #coco
            if split=="val":
                relevance_filenames= [path+"/coco-val-rougeL.npy", path+"/coco-val-spice.npy"] #(5000,1000)
                self.npts=1000 # [:5000] line 60 https://github.com/mesnico/TERN/blob/master/data.py
            else:
                relevance_filenames= [path+"/coco-test-rougeL.npy", path+"/coco-test-spice.npy"] #(25000,5000)
                self.npts=5000 #images
        
        self.relevances = [np.memmap(f, dtype=np.float32, mode='r') for f in relevance_filenames]
        
        # una fila se lleva a una matriz de n_queries filas y lo que quede de columnas (en orden)
        for r in self.relevances:
            r.shape = (self.npts*5, -1) #debe dividir a 5 millones

    def compute_ndcg(self, query_id, sorted_indexes, fold_index=0, retrieval='t2i'):
        """
        sorted_indexe: all sorted indices
        query_id: image retrieval \in [0,5000], sentence retrieval \in [0,1000]
        # npts: images.shape[0] // 5
        fold_index= for 5fold cross-validation, only for MSCOCO
        """
        sorted_indexes = sorted_indexes[:self.rank] #TOP@K
        if retrieval == 't2i':
            query_base = self.npts * 5 * fold_index #0
            #caption query pertenece a  [0,5000] valores de relevancia en vector [0,1000] imagenes
            # (5000,1000)
            relevances = [r[query_base + query_id, fold_index * self.npts : (fold_index + 1) * self.npts] for r in self.relevances]
        elif retrieval == 'i2t':
            query_base = self.npts * fold_index #0
            #  valores de relevancia en vector [0,5000] captions, image query pertenece a  [0,1000] 
            relevances = [r[fold_index * self.npts * 5 : (fold_index + 1) * self.npts * 5, query_base + query_id] for r in self.relevances]
        # por métrica: ndcg(valor de relevancia, posición del ranking) 
        #print(r.shape)
        ndcg_scores = [ ndcg_from_ranking(r, sorted_indexes)  for r in relevances]
        out = {k: v for k, v in zip(self.relevance_methods, ndcg_scores)}
        return out


    


def ndcg_from_ranking(y_true, ranking):

    # El orden del muestreo es irrelevante en ndcg_from_ranking o dcg_score:
    # [0.5, 0.3, 0.2], [2, 1, 0] == [0.2, 0.3, 0.5], [0, 1, 2]
    # lo que importan es el mach (relevancia,posición del ranking)
    
    """Normalized discounted cumulative gain (NDCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    NDCG @k : float
    """
    k = len(ranking)
    best_ranking = np.argsort(y_true)[::-1]#indices de mayor a menor valores
    best = dcg_from_ranking(y_true, best_ranking[:k]) #el mejor posible 
    if best == 0:
        return 0
    return dcg_from_ranking(y_true, ranking) / best

def dcg_from_ranking(y_true, ranking):
    #Por ejemplo en el caso binario:
    #dcg_from_ranking([1, 0, 0], [2, 1, 0])=0.5= 0+0+1/log_2(4)
    #dcg_from_ranking([1, 0, 0], [0, 1, 2])=1.0= 1/log_2(2)+0+0
    #Además, corta solo para el top@K
    """Discounted cumulative gain (DCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels). #comunmente binario
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    DCG @k : float
    """
    y_true = np.asarray(y_true)
    ranking = np.asarray(ranking)
    #ordena y toma el subconjunto de los valores de y_true con respecto al ranking asociado
    rel = y_true[ranking] #normalmente esta es binaria
    gains = 2 ** rel - 1 #en el caso continuo, [0,1] y no {0,1}
    discounts = np.log2(np.arange(len(ranking)) + 2)
    return np.sum(gains / discounts)#div pointwise


#Original https://github.com/mesnico/TERAN/blob/master/evaluation.py


#n_queries= 5000 #img son 1000 files son 5000000==5000*1000


def i2t( model, split, n_ndcg=True, d_n=True, top=25):
    """
    TEXT RETRIEVAL
    
    model: Dado un img query el modelo debe dar el ranking asociado (largo 5000, de 0 a 4999) en orden de aparición!!!
    splt: "val" or "test"
    n_ndcg: novelty ndcg
    d_n: other list metrics diversity and novelty of lists
    """
    output=[]
    
    if split=="val":
      eval_indices= list(val_r_text.keys()) #img
      all_indices= all_val_indices_img #original order  
      eval_values= list(val_r_img.keys()) #text
      all_values= all_val_indices_text #original order
    else:#test
      eval_indices= list(test_r_text.keys()) #img
      all_indices= all_test_indices_img #original order 
      eval_values= list(test_r_img.keys())#text
      all_values= all_test_indices_text #original order

    

    ndcg_scorer = DCG(rank=top, relevance_methods=['rougeL', 'spice'], split=split)
    rougel_ndcgs = np.zeros(ndcg_scorer.npts)
    spice_ndcgs = np.zeros(ndcg_scorer.npts)
    div_rankings=[]
    
    nov_rankings= []
    
    
    for index in range(len(eval_indices)): #range(npts): #each QUERY
        rating= model.predict( universe_img.index(eval_indices[index]) , "i2t", split)      
        if d_n:
            rank_=np.argsort(-rating)
            div_rankings.append( rank_[:top] )
        # positions in order wrt rating (only for evaluation)    
        indices_= np.argsort( -np.array( rating) ) 
        # get original position (in order by the model)
        inds= [all_values.index(eval_values[i]) for i in indices_] #order of values = eval_values
        rougel_ndcgs[index], spice_ndcgs[index] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[index]), inds,retrieval='i2t').values()
        
        if n_ndcg:
            nov_rankings.append(inds)
    
    
    mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
    mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
    output.append( (mean_rougel_ndcg, mean_spice_ndcg) )
    
    if n_ndcg:
        eval_indices_all=[all_indices.index(i) for i in eval_indices]
        rankings={k: v for k, v in zip(eval_indices_all,nov_rankings)}
        
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="rougeL")
        novedad_rougeL=myAlpha.calculate()
        
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="spice")
        novedad_spice=myAlpha.calculate()
        
        output.append((novedad_rougeL,novedad_spice))
    if d_n:
        cos_div,coverage,nov= calculate_diversity(div_rankings, len(eval_indices))
        output.append((coverage,cos_div,nov))
        
    return output


def t2i( model,split,n_ndcg=True, d_n=True, aux_mod=False,top=25):
    #aux_mod for i2i
    output=[]
    if split=="val":
      eval_indices= list(val_r_img.keys()) #text
      all_indices= all_val_indices_text 
      eval_values= len(val_r_text.keys()) #img
      all_values= all_val_indices_img 
    else:
      eval_indices= list(test_r_img.keys())
      all_indices= all_test_indices_text  
      eval_values= list(test_r_text.keys())
      all_values= all_test_indices_img 


    ndcg_scorer = DCG( rank=top, relevance_methods=['rougeL', 'spice'],split=split)
    rougel_ndcgs = np.zeros(5*ndcg_scorer.npts)
    spice_ndcgs = np.zeros(5*ndcg_scorer.npts)
    
    nov_rankings= []
    div_rankings=[]
    
    
    for index in range( len(eval_indices)//5 ): 
        
        for i in range(5): #5 query captions
            if aux_mod==False: #t2i
                rating= model.predict( universe_text.index(eval_indices[5 * index + i]), "t2i", split)
            #i2i
            else:#t->i-> retrieval: each t have a diferent rel wrt each grup of images.
                rating= model.predict( universe_img.index(eval_values[index]) , "i2i",split)
            
            if d_n:
                rank_=np.argsort(-rating)
                div_rankings.append( rank_[:top] )
            
            indices_= np.argsort( -np.array(rating) ) 
            inds= [all_values.index(eval_values[j]) for j in indices_] 
            rougel_ndcgs[5 * index + i], spice_ndcgs[5 * index + i] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[5 * index + i]), inds, retrieval='t2i').values()
            if n_ndcg:
                nov_rankings.append(inds)
            
            
            
    mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
    mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
    output.append((mean_rougel_ndcg, mean_spice_ndcg))
    
    if n_ndcg:
        eval_indices_all=[all_indices.index(i) for i in eval_indices]
        rankings={k: v for k, v in zip( eval_indices_all, nov_rankings)}
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "t2i",ndcg_scorer=ndcg_scorer, method="rougeL")
        novedad_rougeL=myAlpha.calculate()
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "t2i",ndcg_scorer=ndcg_scorer, method="spice")
        novedad_spice=myAlpha.calculate()
        output.append((novedad_rougeL,novedad_spice))
       
    if d_n:
        cos_div,coverage,nov= calculate_diversity(div_rankings, len(eval_indices))
        output.append((coverage,cos_div,nov))  
    return output


def t2t( model, split,n_ndcg=True, d_n=True,top=25):

    output=[]
   
    if split=="val":
      #eval_indices= list(val_r_text.keys()) #img
      all_indices= all_val_indices_img #img 
      all= all_val_indices_text 
      eval_dic= val_r_img
    else:
      #eval_indices= list(test_r_text.keys()) #img
      all_indices= all_test_indices_img #img 
      all= all_test_indices_text
      eval_dic= test_r_img
      
    eval_values= list(eval_dic.keys()) # texts
    

    ndcg_scorer = DCG( rank=top, relevance_methods=['rougeL', 'spice'], split=split)
    rougel_ndcgs = np.zeros(5*ndcg_scorer.npts)
    spice_ndcgs = np.zeros(5*ndcg_scorer.npts)
    querys=[]
    
    nov_rankings= []
    div_rankings=[]
    for index in range( len(eval_values)): 
        rating= model.predict( universe_text.index(eval_values[index]) , "t2t",split)
        if d_n:
            rank_=np.argsort(-rating)
            div_rankings.append( rank_[:top] )
        
        
        indices_= np.argsort( -np.array(rating) ) 
        inds= [all.index(eval_values[i]) for i in indices_] 
        #t->i->index
        img_query_from_text=all_indices.index(eval_dic[eval_values[index]]) 
        querys.append(img_query_from_text)
        rougel_ndcgs[index], spice_ndcgs[index] = ndcg_scorer.compute_ndcg( img_query_from_text, inds,retrieval='i2t').values()
        if n_ndcg:
            nov_rankings.append(inds)
    
        
    mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
    mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
    output.append((mean_rougel_ndcg, mean_spice_ndcg))
    
    if n_ndcg:
        rankings={k: v for k, v in zip(querys,nov_rankings)}
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="rougeL")
        novedad_rougeL=myAlpha.calculate()
        myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="spice")
        novedad_spice=myAlpha.calculate()
        output.append((novedad_rougeL,novedad_spice))
        
    if d_n:
        cos_div,coverage,nov= calculate_diversity(div_rankings, len(eval_values))
        output.append((coverage,cos_div,nov)) 
    return output



In [88]:
class AlphaNDCG(object):
 def __init__(self, alpha=0.5, top=25,rankings=None,  retrieval= "i2t",ndcg_scorer=None, method= None):
  self.alpha = alpha
  self.fold_index=0
  self.retrieval= retrieval
  self.all_rankings= rankings
  self.ndcg_scorer= ndcg_scorer
  self.mode= ['rougeL', 'spice'].index(method)
  self.top= top
  self.npts= self.ndcg_scorer.npts


 def relevance(self, query_id):
    

    if self.retrieval == 't2i':
                query_base = self.npts * 5 * self.fold_index #0
                relevances = [r[query_base + query_id, self.fold_index * self.npts : (self.fold_index + 1) * self.npts] for r in self.ndcg_scorer.relevances]
    if self.retrieval == 'i2t':
                query_base = self.npts * self.fold_index #0
                relevances = [r[self.fold_index * self.npts * 5 : (self.fold_index + 1) * self.npts * 5, query_base + query_id] for r in self.ndcg_scorer.relevances]

    return relevances[self.mode]


 def top_querys(self, query_id,querys_a_ordenar):   

    if self.retrieval == 't2i':
        value_retrieval= "i2t" #value=img pasa a ser query para retrieval de text
    if self.retrieval == 'i2t':
        value_retrieval= "t2i"
        
    if value_retrieval == 't2i': #retrieval img quer
                query_base = self.npts * 5 * self.fold_index #0
                relevances = [r[query_base + query_id, self.fold_index * self.npts : (self.fold_index + 1) * self.npts] for r in self.ndcg_scorer.relevances]
    if value_retrieval == 'i2t':
                query_base = self.npts * self.fold_index #0
                relevances = [r[self.fold_index * self.npts * 5 : (self.fold_index + 1) * self.npts * 5, query_base + query_id] for r in self.ndcg_scorer.relevances]

                
    return sorted(querys_a_ordenar, key=lambda x: list(relevances[self.mode])[x], reverse= True)[:self.top] 





 def ideal_value(self):
   # normalizarion with top most relevant per query 
  ndcg=0
  for query in self.all_rankings: # each ranking
   ranking= np.argsort(-self.relevance(query))[:self.top] #top by relevance
   metric=0
   i=1

   for value in ranking: #each value
        if i==1: #first 
            sum_numerator=self.relevance(query)[value]
            rel_ac= self.relevance(query)[value]
        else:
            sum_numerator+=self.relevance(query)[value]  * ( (1 - self.alpha)**rel_ac )
            rel_ac+= self.relevance(query)[value]

        metric+= sum_numerator/np.log2(i+1)
        i+=1
      
   ndcg+=metric

  return ndcg/ len(self.all_rankings) #mean



 def calculate(self):
  ndcg=0
  normalization= self.ideal_value()
    
  for query in self.all_rankings: # each ranking
    ranking= self.all_rankings[query][:self.top]
    metric=0
    i=1
    for value in ranking: #each value
     if i==1:
        sum_numerator=self.relevance(query)[value]
        rel_ac= self.relevance(query)[value]
     else:
        sum_numerator+=self.relevance(query)[value]  * ( (1 - self.alpha)**rel_ac )
        rel_ac+= self.relevance(query)[value]
         
     metric+= sum_numerator/np.log2(i+1)
     i+=1
      
    ndcg+=metric/ normalization

  return ndcg/ len(self.all_rankings) #mean


     

# ALIGN

In [89]:
class align(object):
    def __init__(self):
            print("ok")
            
    def predict(self,query, retrieval,split):
            if type(query) is not list:
                query= [query]
            if retrieval=="t2i":
                c_input = tf.nn.embedding_lookup(textual_matrix, query)
                if split=="test":
                    i_eval= visual_matrix[self.test_indices_]
                
                rating= tf.matmul(i_eval, tf.transpose(c_input))
            
            if retrieval=="i2t": 
                i_input = tf.nn.embedding_lookup(visual_matrix, query)
                if split=="test":
                    c_eval= textual_matrix[self.data_text_Test]            
        
                rating= tf.matmul(i_input, tf.transpose(c_eval) )

            if retrieval=="i2i":
                if split=="test":
                    i_query = tf.nn.embedding_lookup(visual_matrix, query)
                    i_eval=  visual_matrix[self.test_indices_]
                    rating= tf.matmul(i_query, tf.transpose(i_eval) )
            if retrieval=="t2t":
                if split=="test":
                    c_query = tf.nn.embedding_lookup(textual_matrix, query)
                    c_eval= textual_matrix[self.data_text_Test]
                    rating= tf.matmul(c_eval , tf.transpose(c_query)  )

            return np.reshape(rating, [-1]) 
        
    def inference(self):
        self.test_indices_= [universe_img.index(i) for i in test_r_text.keys()]
        self.data_text_Test= [universe_text.index(i) for i in test_r_img.keys()]

        print("inference")
        results= evaluate_model(self, nDCG=True, split="test", retrieval="i2t")
        #results= evaluate_model(self, nDCG=False, split="test", retrieval="i2t")
        #print("resultado: ", results)
        #results= evaluate_model(self, nDCG=False, split="test", retrieval="t2i")
        print("resultado: ", results)
        #np.save("ALIGN/intramodal_ALIGN_F30K.npy", np.array([results], dtype=object) )

In [90]:
net =  align() 
net.inference()

ok
inference
novedad:  25 25000
novedad:  25 25000
resultado:  [([(0.6994408953769155, 0.5930526836428482), (0.931029768488452, 0.768424797126483), (0.0, 0.0, 6.861907868792291)], [(0.3333938055178477, 0.2513052554786508), (0.6581775352535136, 0.44547320179654387), (0.0, 0.0, 3.9804621908242166)])]


**ALIGN_coco**

i2t:

resultado:  [[(0.5798541684164804, 0.549496780658785), (0.8498801240862058, 0.7635262730207658), (0.0, 0.0, 8.814685769550442)], [[52.34, 76.18, 84.34]]]


t2i:

novedad:  25 25000
resultado:  [[(0.6708697756247558, 0.6083469475949774), (0.8578689903535616, 0.8579067180803532), (0.0, 0.0, 7.305718684389171)], [[40.14, 65.84, 75.592]]]


inference
novedad:  25 25000
novedad:  25 25000
resultado:  [([(0.6994408953769155, 0.5930526836428482), (0.931029768488452, 0.768424797126483), (0.0, 0.0, 6.861907868792291)], [(0.3333938055178477, 0.2513052554786508), (0.6581775352535136, 0.44547320179654387), (0.0, 0.0, 3.9804621908242166)])]

**ALIGN_F30K**

i2t:

resultado:  [[(0.7184695601060688, 0.6300024630569074), (0.9326004925070543, 0.8265177202472233), (0.0, 0.0, 7.131597458850054)], [[80.9, 95.6, 98.3]]]

t2i:

resultado:  [[(0.7126037626733526, 0.6083792966951824), (0.9035168985879684, 0.7542624440346034), (0.0, 0.0, 5.09503008054014)], [[71.46, 91.88, 95.5]]]

i2i and t2t:


novedad:  25 5000
novedad:  25 5000
resultado:  [(0, 0, [(0.7037359014719643, 0.5770470107111444), (0.9263472349685231, 0.7193908739450162), (0.0, 0.0, 4.559317138252759)], [(0.5575308739982041, 0.4184562318675485), (0.8602634738724274, 0.5461367525717573), (0.0, 0.0, 4.667067951674062)])]